## Namespace

In [ ]:
%%writefile namespace.yaml
apiVersion: v1
kind: Namespace
metadata:
  name: airflow

## Install

In [1]:
%%writefile install.sh
#!/bin/bash

microk8s helm3 repo add apache-airflow https://airflow.apache.org
microk8s helm3 repo update
microk8s kubectl create -f namespace.yaml

microk8s helm3 install airflow apache-airflow/airflow \
--namespace airflow \
--set executor=CeleryKubernetesExecutor

Overwriting install.sh


## DAG

In [ ]:
%%writefile test_dag.py
from datetime import datetime, timedelta

from airflow.models.dag import DAG
from airflow.operators.dummy import DummyOperator
from airflow.utils.dates import days_ago

now = datetime.now()
now_to_the_hour = (now - timedelta(0, 0, 0, 0, 0, 3)).replace(minute=0, second=0, microsecond=0)
START_DATE = now_to_the_hour
DAG_NAME = 'test_dag_v1'

default_args = {'owner': 'airflow', 'depends_on_past': True, 'start_date': days_ago(1)}
dag = DAG(DAG_NAME, schedule_interval='0 0 * * *', default_args=default_args)

run_this_1 = DummyOperator(task_id='run_this_1', dag=dag)
run_this_2 = DummyOperator(task_id='run_this_2', dag=dag)
run_this_2.set_upstream(run_this_1)
run_this_3 = DummyOperator(task_id='run_this_3', dag=dag)
run_this_3.set_upstream(run_this_2)

## Docker

In [ ]:
%%writefile requirements.txt
apache-airflow-providers-microsoft-mssql
apache-airflow-providers-mysql
apache-airflow-providers-neo4j
apache-airflow-providers-apache-beam
apache-airflow-providers-google

In [ ]:
%%writefile Dockerfile
From apache/airflow:2.2.1-python3.8

USER root
RUN apt-get update -yqq \
    && apt-get install -yqq --no-install-recommends \
        build-essential \
    && apt-get upgrade -yqq \
    && apt-get autoremove --purge -yqq \
    && apt-get autoclean -yqq \
    && rm -rf /var/lib/apt/lists/*

COPY --chown=airflow:root test_dag.py /opt/airflow/dags/

USER airflow
COPY requirements.txt .
RUN python3 -m pip install --no-cache-dir -qU pip wheel setuptools \
    && python3 -m pip install --no-cache-dir -Ur requirements.txt \
    && rm -rf requirements.txt

In [ ]:
%%writefile build.sh
#!/bin/bash

docker build . --no-cache \
-f Dockerfile \
-t "darklemon/test-airflow-dag:0.0.1"

docker push darklemon/test-airflow-dag:0.0.1

In [ ]:
%%writefile upgrade.sh
#!/bin/bash

microk8s helm3 upgrade airflow apache-airflow/airflow \
--namespace airflow \
--set images.airflow.repository=darklemon/test-airflow-dag \
--set images.airflow.tag=0.0.1 \
--set images.airflow.pullPolicy=Always

## Uninstall

In [ ]:
%%writefile uninstall.sh
#!/bin/bash

microk8s helm3 delete airflow \
--namespace airflow
microk8s kubectl delete -f namespace.yaml
microk8s helm3 repo rm apache-airflow